In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
def get_html(url: str):
    res = requests.get(url)
    return res.text

In [4]:
page_url = "https://pt.wikipedia.org/wiki/Lista_de_praias_de_Pernambuco"

html = get_html(page_url)
soup = BeautifulSoup(html, 'lxml')

In [29]:
cities = soup.select(".mw-parser-output>h2>span.mw-headline:not(#Galeria,#Referências)")
cities = [(city.text, city.get("id")) for city in cities]
cities

[('Goiana', 'Goiana'),
 ('Ilha de Itamaracá', 'Ilha_de_Itamaracá'),
 ('Igarassu', 'Igarassu'),
 ('Paulista', 'Paulista'),
 ('Olinda', 'Olinda'),
 ('Recife', 'Recife'),
 ('Jaboatão', 'Jaboatão'),
 ('Cabo de Santo Agostinho', 'Cabo_de_Santo_Agostinho'),
 ('Ipojuca', 'Ipojuca'),
 ('Sirinhaém', 'Sirinhaém'),
 ('Rio Formoso', 'Rio_Formoso'),
 ('Tamandaré', 'Tamandaré'),
 ('Barreiros', 'Barreiros'),
 ('São José da Coroa Grande', 'São_José_da_Coroa_Grande'),
 ('Ilhas de Fernando de Noronha', 'Ilhas_de_Fernando_de_Noronha')]

In [74]:
tags = soup.select(f".mw-parser-output > *:is(h2, ul)")

grouped = []
group = None
for tag in tags:
    if tag.name == "h2":
        if tag.select_one(".mw-headline").get("id") == "Galeria":
            break

        city = tag.select(".mw-headline")
        assert city, "City not found"
        city = city[0].text
        if not group:
            group = [city, []]
        else:
            grouped.append(group)
            group = [city, []]
    else:
        beaches = tag.select("li")
        beaches = [beach.text for beach in beaches]
        group[1].extend(beaches)

grouped.append(group)

In [76]:
import pandas as pd

data = []

for city, beaches in grouped:
    for beach in beaches:
        data.append({
            'city': city,
            'beach': beach
        })

pd.DataFrame(data)

,city,beach
0,Goiana,Carne de Vaca
1,Goiana,Pontas de Pedras
2,Goiana,Barra de Catuama
3,Goiana,catuama
4,Goiana,Atapuz
...,...,...
86,Ilhas de Fernando de Noronha,Atalaia
87,Ilhas de Fernando de Noronha,Enseada da Caeira
88,Ilhas de Fernando de Noronha,Buraco da Raquel
89,Ilhas de Fernando de Noronha,Caeira
